In [21]:
# LangChain components to use
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
# Support for dataset retrieval with Hugging Face
from datasets import load_dataset

# With CassIO, the engine powering the Astra DB integration in LangChain,
# you will also initialize the DB connection:
import cassio

In [10]:
from PyPDF2 import PdfReader

### Setup

In [26]:
load_dotenv()

astra_db_token = os.getenv("astra_db_token")
astra_db_id = os.getenv("astra_db_id")
groq_api_key = os.getenv("groq_api_key")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "your_huggingface_api_key_here"

In [15]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('notes.pdf')

In [16]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [17]:
raw_text

' \n1. **Introduction to Machine Learning**   \n   Machine Learning (ML) is a branch of artificial intelligence that focuses on building \nsystems capable of learning from data. Rather than being explicitly programmed for \nevery task, ML models are trained on data to recognize patterns, make decisions, and  \neven predict future outcomes. It has transformed many industries by enabling \ncomputers to improve their performance on specific tasks over time.  \n \n2. **How Machine Learning Works**   \n   ML systems learn by analyzing vast amounts of data. They identify patterns, \ncorrelations, and trends, allowing them to make predictions or decisions without \nhuman intervention. This learning process typically involves feeding data into \nalgorithms that a djust themselves based on the data patterns they observe, refining \ntheir accuracy with more data over time.  \n \n3. **Types of Machine Learning**   \n   Machine Learning is generally categorized into three main types: supervised, \

Initialize the connection to your database:

_(do not worry if you see a few warnings, it's just that the drivers are chatty about negotiating protocol versions with the DB.)_

In [18]:
cassio.init(token=astra_db_token, database_id=astra_db_id)

In [28]:
llm = ChatGroq(groq_api_key = groq_api_key,model_name = "Llama3-8b-8192")
embeddings = HuggingFaceEmbeddings(model = "all-MiniLM-L6-v2")

ValidationError: 1 validation error for HuggingFaceEmbeddings
model
  Extra inputs are not permitted [type=extra_forbidden, input_value='all-MiniLM-L6-v2', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/extra_forbidden

Create your LangChain vector store ... backed by Astra DB!

In [ ]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="notes",
    session=None,
    keyspace=None,
)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
texts[:50]

['GOVERNMENT OF INDIA\nBUDGET 2023-2024\nSPEECH\nOF\nNIRMALA SITHARAMAN\nMINISTER OF FINANCE\nFebruary 1,  2023CONTENTS \nPART-A \n Page No.  \n\uf0b7 Introduction 1 \n\uf0b7 Achievements since 2014: Leaving no one behind 2 \n\uf0b7 Vision for Amrit Kaal  – an empowered and inclusive economy 3 \n\uf0b7 Priorities of this Budget 5 \ni. Inclusive Development  \nii. Reaching the Last Mile \niii. Infrastructure and Investment \niv. Unleashing the Potential \nv. Green Growth \nvi. Youth Power  \nvii. Financial Sector  \n \n \n \n \n \n \n \n \n\uf0b7 Fiscal Management 24 \nPART B  \n  \nIndirect Taxes  27 \n\uf0b7 Green Mobility  \n\uf0b7 Electronics   \n\uf0b7 Electrical   \n\uf0b7 Chemicals and Petrochemicals   \n\uf0b7 Marine products  \n\uf0b7 Lab Grown Diamonds  \n\uf0b7 Precious Metals  \n\uf0b7 Metals  \n\uf0b7 Compounded Rubber  \n\uf0b7 Cigarettes  \n  \nDirect Taxes  30 \n\uf0b7 MSMEs and Professionals',
 '\uf0b7 Chemicals and Petrochemicals   \n\uf0b7 Marine products  \n\uf0b7 La

### Load the dataset into the vector store



In [ ]:

astra_vector_store.add_texts(texts[:50])

print("Inserted %i headlines." % len(texts[:50]))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 50 headlines.


In [ ]:
first_question = True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("FIRST DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))


Enter your question (or type 'quit' to exit): How much the agriculture target will be increased to and what the focus will be

QUESTION: "How much the agriculture target will be increased to and what the focus will be"


ANSWER: "The agriculture credit target will be increased to ` 20 lakh crore with focus on animal husbandry, dairy and fisheries."

FIRST DOCUMENTS BY RELEVANCE:


    [0.9327] "of Millet Research, Hyderabad  will be supported as the Centre of Excellence 
for sh ..."
    [0.9327] "of Millet Research, Hyderabad  will be supported as the Centre of Excellence 
for sh ..."
    [0.9207] "Agriculture Accelerator Fund  
17. An Agriculture Accelerator Fund will be set-up to ..."
    [0.9207] "Agriculture Accelerator Fund  
17. An Agriculture Accelerator Fund will be set-up to ..."

What's your next question (or type 'quit' to exit): What is the current GDP

QUESTION: "What is the current GDP"


ANSWER: "The current GDP is estimated to be 7 per cent."

FIRST DOCUMENTS BY RELEVANCE:


    [0.8920] "estimated to be at 7 per cent. It is notable that this is the highest among all 
the ..."
    [0.8920] "estimated to be at 7 per cent. It is notable that this is the highest among all 
the ..."
    [0.8915] "multiplier impact on growth and employment. After the subdued period of 
the pandemi ..."
    [0.8915] "multiplier impact on growth and employment. After the subdued period of 
the pandemi ..."

What's your next question (or type 'quit' to exit): quit
